# Pre-Processing and Training Data
Create a cleaned development dataset you can use for modeling.

* Create dummy or indicator features for categorical variables
    * pd.get_dummies()
* 
Standardize the magnitude of numeric features using a scaer
    * scaler = preprocessing.StandardScaler() (make a Scaler object)
    * scaled_df = scaler.fit_transform(df) (fit data to the scaler object)
    * scaled_df = pd.DataFrame(scaled_df, columns=names)
* 
Split your data into testing and training datase
    * sklearn functions (train_test_split())

### Questions:
* What categorical data do I have?
* Do the features have values that range from 0-100 or 0-1?ts

# 1.) Load libraries and data

In [136]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
pd.set_option("display.max_rows", 10)

In [137]:
# Load data
allDogs = pd.read_csv('clean2- allDogDescription.csv')
dogTravel = pd.read_csv('clean2- dogTravel.csv')
movesByLocation = pd.read_csv('clean2- movesByLocation.csv')

C:\Users\yenmi\AppData\Local\Temp\ipykernel_12900\2673621457.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  allDogs = pd.read_csv('clean2- allDogDescription.csv')


In [138]:
allDogs.head()
# dogTravel.head()
# movesByLocation.head()

,org_id,breed_primary,breed_mixed,color_primary,age,sex,size,coat,fixed,house_trained,special_needs,shots_current,env_children,env_dogs,name,contact_city,contact_state,contact_zip,contact_country,region,description
0,NV163,American Staffordshire Terrier,True,White / Cream,Senior,Male,Medium,Short,True,True,False,True,True,True,HARLEY,Las Vegas,NV,89147,US,West,Harley is not sure how he wound up at shelter ...
1,NV163,Pit Bull Terrier,True,Brown / Chocolate,Adult,Male,Large,Short,True,True,False,True,True,True,BIGGIE,Las Vegas,NV,89147,US,West,6 year old Biggie has lost his home and really...
2,NV99,Shepherd,False,Brindle,Adult,Male,Large,Short,True,False,False,True,True,True,Ziggy,Mesquite,NV,89027,US,West,Approx 2 years old.\n Did I catch your eye? I ...
3,NV202,German Shepherd Dog,False,NaN,Baby,Female,Large,NaN,False,False,False,False,True,True,Gypsy,Pahrump,NV,89048,US,West,NaN
4,NV184,Dachshund,False,NaN,Young,Male,Small,Long,True,False,False,True,True,True,Theo,Henderson,NV,89052,US,West,Theo is a friendly dachshund mix who gets alon...


# 2.) Create dummy variables

## allDogs

In [141]:
# Not all categories should have a dummy variable. Some of them have too many unique categories
for col in allDogs.columns:
    print(col + " unique values: " + str(allDogs[col].nunique()))

org_id unique values: 3969
breed_primary unique values: 216
breed_mixed unique values: 2
color_primary unique values: 15
age unique values: 4
sex unique values: 3
size unique values: 4
coat unique values: 6
fixed unique values: 2
house_trained unique values: 2
special_needs unique values: 2
shots_current unique values: 2
env_children unique values: 2
env_dogs unique values: 2
name unique values: 22953
contact_city unique values: 2176
contact_state unique values: 53
contact_zip unique values: 3537
contact_country unique values: 2
region unique values: 5
description unique values: 46289


In [142]:
# HOWEVER: some can be categorized as "Other", no?
# EXPLORE LATER

In [143]:
# Create function that returns list of categories to be one-hot encoded
def identify_dummy_categories(df, thresh=10):
    categories = []
    for col in df.columns:
        if df[col].nunique() < thresh:
            categories.append(col)
            print(col + " unique values: " + str(df[col].nunique()))
    return categories

In [144]:
identify_dummy_categories(allDogs)

breed_mixed unique values: 2
age unique values: 4
sex unique values: 3
size unique values: 4
coat unique values: 6
fixed unique values: 2
house_trained unique values: 2
special_needs unique values: 2
shots_current unique values: 2
env_children unique values: 2
env_dogs unique values: 2
contact_country unique values: 2
region unique values: 5


['breed_mixed',
 'age',
 'sex',
 'size',
 'coat',
 'fixed',
 'house_trained',
 'special_needs',
 'shots_current',
 'env_children',
 'env_dogs',
 'contact_country',
 'region']

In [145]:
allDogs_encoded = pd.get_dummies(allDogs, columns=identify_dummy_categories(allDogs))
allDogs_encoded.head()

breed_mixed unique values: 2
age unique values: 4
sex unique values: 3
size unique values: 4
coat unique values: 6
fixed unique values: 2
house_trained unique values: 2
special_needs unique values: 2
shots_current unique values: 2
env_children unique values: 2
env_dogs unique values: 2
contact_country unique values: 2
region unique values: 5


,org_id,breed_primary,color_primary,name,contact_city,contact_state,contact_zip,description,breed_mixed_False,breed_mixed_True,age_Adult,age_Baby,age_Senior,age_Young,sex_Female,sex_Male,sex_Unknown,size_Extra Large,size_Large,size_Medium,size_Small,coat_Curly,coat_Hairless,coat_Long,coat_Medium,coat_Short,coat_Wire,fixed_False,fixed_True,house_trained_False,house_trained_True,special_needs_False,special_needs_True,shots_current_False,shots_current_True,env_children_False,env_children_True,env_dogs_False,env_dogs_True,contact_country_CA,contact_country_US,region_Canada,region_Midwest,region_Northeast,region_South,region_West
0,NV163,American Staffordshire Terrier,White / Cream,HARLEY,Las Vegas,NV,89147,Harley is not sure how he wound up at shelter ...,False,True,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,True,False,True,True,False,False,True,False,True,False,True,False,True,False,False,False,False,True
1,NV163,Pit Bull Terrier,Brown / Chocolate,BIGGIE,Las Vegas,NV,89147,6 year old Biggie has lost his home and really...,False,True,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,True,False,True,True,False,False,True,False,True,False,True,False,True,False,False,False,False,True
2,NV99,Shepherd,Brindle,Ziggy,Mesquite,NV,89027,Approx 2 years old.\n Did I catch your eye? I ...,True,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,True,True,False,True,False,False,True,False,True,False,True,False,True,False,False,False,False,True
3,NV202,German Shepherd Dog,NaN,Gypsy,Pahrump,NV,89048,NaN,True,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False,True,False,True,False,False,True,False,True,False,True,False,False,False,False,True
4,NV184,Dachshund,NaN,Theo,Henderson,NV,89052,Theo is a friendly dachshund mix who gets alon...,True,False,False,False,False,True,False,True,False,False,False,False,True,False,False,True,False,False,False,False,True,True,False,True,False,False,True,False,True,False,True,False,True,False,False,False,False,True


## dogTravel

In [147]:
for col in dogTravel.columns:
    print(col + " unique values: " + str(dogTravel[col].nunique()))

id unique values: 4115
contact_city unique values: 634
contact_state unique values: 44
contact_region unique values: 4
description unique values: 3926
found unique values: 86
found_region unique values: 5


In [148]:
identify_dummy_categories(dogTravel)

contact_region unique values: 4
found_region unique values: 5


['contact_region', 'found_region']

In [149]:
dogTravel_encoded = pd.get_dummies(dogTravel, columns=identify_dummy_categories(dogTravel))
dogTravel_encoded.head()

contact_region unique values: 4
found_region unique values: 5


,id,contact_city,contact_state,description,found,contact_region_Midwest,contact_region_Northeast,contact_region_South,contact_region_West,found_region_International,found_region_Midwest,found_region_Northeast,found_region_South,found_region_West
0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,AR,True,False,False,False,False,False,False,True,False
1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Bahamas,False,False,True,False,True,False,False,False,False
2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,MD,False,False,True,False,False,False,False,True,False
3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,MN,True,False,False,False,False,True,False,False,False
4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,False,False,False,True,True,False,False,False,False


## movesByLocation

In [151]:
for col in movesByLocation.columns:
    print(col + " unique values: " + str(movesByLocation[col].nunique()))

location unique values: 88
region unique values: 11
exported unique values: 34
imported unique values: 30
total unique values: 49
inUS unique values: 2


In [152]:
identify_dummy_categories(movesByLocation, 12)

region unique values: 11
inUS unique values: 2


['region', 'inUS']

movesByLocation_encoded = pd.get_dummies(movesByLocation, columns=identify_dummy_categories(movesByLocation, 12))
movesByLocation_encoded.head()

# 3.) Standardize values

In [155]:
pd.set_option("display.max_columns", None)
allDogs_encoded.head() # nothing to standardize
dogTravel_encoded.head() # nothing to standardize
movesByLocation_encoded.head() # Can standardize 'exported', 'imported', and 'total'

,location,exported,imported,total,region_Asia,region_Caribbean,region_Central America,region_Europe,region_Middle East,region_Midwest,region_North America,region_Northeast,region_South,region_South America,region_West,inUS_False,inUS_True
0,TX,635.0,NaN,566.0,False,False,False,False,False,False,False,False,True,False,False,False,True
1,AL,268.0,2.0,1428.0,False,False,False,False,False,False,False,False,True,False,False,False,True
2,NC,158.0,14.0,2627.0,False,False,False,False,False,False,False,False,True,False,False,False,True
3,SC,139.0,12.0,1618.0,False,False,False,False,False,False,False,False,True,False,False,False,True
4,GA,137.0,19.0,3479.0,False,False,False,False,False,False,False,False,True,False,False,False,True


In [156]:
scaler = StandardScaler()

In [157]:
movesByLocation_scaled = movesByLocation_encoded.copy()

In [158]:
features = ['exported', 'imported', 'total']
movesByLocation_scaled[features] = scaler.fit_transform(movesByLocation_scaled[features])
movesByLocation_scaled

,location,exported,imported,total,region_Asia,region_Caribbean,region_Central America,region_Europe,region_Middle East,region_Midwest,region_North America,region_Northeast,region_South,region_South America,region_West,inUS_False,inUS_True
0,TX,7.374850,NaN,-0.571224,False,False,False,False,False,False,False,False,True,False,False,False,True
1,AL,2.893022,-0.674824,0.286871,False,False,False,False,False,False,False,False,True,False,False,False,True
2,NC,1.549695,-0.545747,1.480440,False,False,False,False,False,False,False,False,True,False,False,False,True
3,SC,1.317666,-0.567260,0.476011,False,False,False,False,False,False,False,False,True,False,False,False,True
4,GA,1.293242,-0.491965,2.328581,False,False,False,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,DE,NaN,-0.384401,-0.840001,False,False,False,False,False,False,False,False,True,False,False,False,True
84,MI,NaN,-0.567260,-0.464709,False,False,False,False,False,True,False,False,False,False,False,False,True
85,Washington DC,NaN,-0.578017,-0.800183,False,False,False,False,False,False,False,False,False,False,False,False,True
86,MT,NaN,NaN,-1.116742,False,False,False,False,False,False,False,False,False,False,True,False,True


# 4.) Split into test and training data

In [208]:
# allDogs_encoded.head()
dogTravel_encoded.head()
# movesByLocation_scaled.head()

,id,contact_city,contact_state,description,found,contact_region_Midwest,contact_region_Northeast,contact_region_South,contact_region_West,found_region_International,found_region_Midwest,found_region_Northeast,found_region_South,found_region_West
0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,AR,True,False,False,False,False,False,False,True,False
1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Bahamas,False,False,True,False,True,False,False,False,False
2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,MD,False,False,True,False,False,False,False,True,False
3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,MN,True,False,False,False,False,True,False,False,False
4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,False,False,False,True,True,False,False,False,False


In [210]:
# X is all the columns except for 1 target variable
# y is the target variable
# But what exactly are we predicting?

# By using the 'dogTravel_encoded'
# We can predict to this questions: "Where might have this dog come from, given where it is now?"
target = 'found'
X = dogTravel_encoded.drop(columns=target, inplace=False)
y = dogTravel_encoded[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 5.) Save updated csv files

In [162]:
allDogs_encoded.to_csv('encoded-allDogDescription.csv', index=False)
dogTravel_encoded.to_csv('encoded-dogTravel.csv', index=False)
movesByLocation_encoded.to_csv('encoded-movesByLocation.csv', index=False)